In [ ]:
import pandas as pd 

df = pd.read_csv("news_body_new.tsv", sep="\t")

corpus = []
for _, row in df.iterrows():
    # corpus = [p.replace("\n", "") for p in row["body"]]
    corpus.append(row["body"].replace("\n", ""))

corpus

In [ ]:

m1 = 'msmarco-distilbert-cos-v5'
m2 = 'all-mpnet-base-v2'
m3 = 'multi-qa-mpnet-base-dot-v1'
m4 = 'nq-distilbert-base-v1'
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer(m4)

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)


In [ ]:
queries = ['Eastern Insurance Company Ltd.']
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """

In [ ]:
import glob
from pyresparser import ResumeParser

path = "/home/parvej/Desktop/Machine Learning Engineer_Mar 29'23"
files = glob.glob(f"{path}/*.pdf")
print(files[0])
from resume_parser import resumeparse

data = resumeparse.read_file(files[0])

In [ ]:
from pdfminer.high_level import extract_text
import os 
from pathlib import Path

for file in files:
    text = extract_text(file)

    with open(Path("cvs")/os.path.basename(file).replace(".pdf", ".txt"), "w") as f:
        f.write(text)
    print(text)

In [ ]:
from io import StringIO
from pdfminer import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
pdf_path = files[0]
def to_txt(pdf_path):
    input_ = file(pdf_path, 'rb')
    output = StringIO()

    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    process_pdf(manager, converter, input_)

    return output.getvalue() 

In [2]:
import os
import glob
import ast 
import pandas as pd 

with open("responses.txt", "r") as f:
    l = [ast.literal_eval(p) for p in f.readlines()]

ls = []
new_l = {}
for a in l:
    # if a["id"] == '1bc32b9fc4ca4f288c8b847de4f9b6df':
    #     continue
    try:
        r = a['response']
        # print(r[0], r[-1])
        if r[0] == "[" and r[-1] == "]":
            r = r.replace("[", "").replace("]", "")
            print(r)
            # new_l = {}
            # new_l["id"] = a["id"]
            # new_l["c_names"] = ast.literal_eval(str(map(str, r.split(","))))
            new_l[a["id"]] = [p.replace("\n", "").replace("Limited", "Ltd.").lstrip() for p in r.split(",")]

            
    except Exception as e:
        print(str(e))

new_l



Shahjibazar Power Co. Ltd., MJL Bangladesh Ltd., Doreen Power Generations & Systems Ltd., Khan Brothers PP Woven Bag Industries Ltd., Familytex (BD) Ltd., Orion Infusions Ltd., Lafarge Surma Cement Ltd., Grameenphone Ltd., British American Tobacco Bangladesh Company Ltd., Berger Paints Bangladesh Ltd., Linde Bangladesh Ltd., and Heidelberg Cement Bangladesh Ltd.
No specific companies listed as the article focuses on the overall market performance
ACI Ltd., Beximco Pharmaceuticals Ltd., United Power Generation and Distribution Company Ltd., Lafarge Surma Cement Ltd., Standard Ceramics Ltd., First Bangladesh Fixed Income Fund
Bangladesh Securities and Exchange Commission (BSEC)
Square Pharmaceuticals Ltd., Lafarge Surma Cement Ltd.,                Shahjibazar Power Co. Ltd., Bangladesh Building Systems Ltd., Islami Bank Bangladesh Ltd.,                Al-Haj Textile Mills Ltd., ICB Second Mutual Fund
MJL Bangladesh Limited, National Tubes Limited, Bangladesh Steel Re-rolling Mills, Far C

{'1bc32b9fc4ca4f288c8b847de4f9b6df': ['Shahjibazar Power Co. Ltd.',
  'MJL Bangladesh Ltd.',
  'Doreen Power Generations & Systems Ltd.',
  'Khan Brothers PP Woven Bag Industries Ltd.',
  'Familytex (BD) Ltd.',
  'Orion Infusions Ltd.',
  'Lafarge Surma Cement Ltd.',
  'Grameenphone Ltd.',
  'British American Tobacco Bangladesh Company Ltd.',
  'Berger Paints Bangladesh Ltd.',
  'Linde Bangladesh Ltd.',
  'and Heidelberg Cement Bangladesh Ltd.'],
 'c65d3f79f73643f29bcff9ed07af7529': ['No specific companies listed as the article focuses on the overall market performance'],
 'f6d98dbd22714e96aa3dc762c8ba83fc': ['ACI Ltd.',
  'Beximco Pharmaceuticals Ltd.',
  'United Power Generation and Distribution Company Ltd.',
  'Lafarge Surma Cement Ltd.',
  'Standard Ceramics Ltd.',
  'First Bangladesh Fixed Income Fund'],
 '99f319876f784688a6fe65ad945aabc3': ['Bangladesh Securities and Exchange Commission (BSEC)'],
 '869c1f312dab4860a15bbff731ce1433': ['Square Pharmaceuticals Ltd.',
  'Lafarge Sur

In [3]:
from fuzzywuzzy import fuzz
with open("comapany_names.txt", "r") as f:
    p = [n for n in f]

q = {}
for a in p:
    tick, name = a.split("|")
    q[name.strip().replace('Limited', 'Ltd.')] = tick

q

{'Aamra Networks Ltd.': 'AAMRANET ',
 'Aamra Technologies Ltd.': 'AAMRATECH ',
 'Ab Bank 1St Mutual Fund': 'ABB1STMF ',
 'Ab Bank Ltd.': 'ABBANK ',
 'Aman Cotton Fibrous Ltd.': 'ACFL ',
 'Aci  Ltd.': 'ACI ',
 'Aci Formulations Ltd.': 'ACIFORMULA ',
 'The Acme Laboratories Ltd.': 'ACMELAB ',
 'Acme Pesticides Ltd.': 'ACMEPL ',
 'Active Fine Chemicals Ltd.': 'ACTIVEFINE ',
 'Adn Telecom Ltd.': 'ADNTEL ',
 'Advent Pharma Ltd.': 'ADVENT ',
 'Afc Agro Biotech Ltd.': 'AFCAGRO ',
 'Aftab Automobiles Ltd.': 'AFTABAUTO ',
 'Agni Systems Ltd.': 'AGNISYSL ',
 'Agrani Insurance Co. Ltd.': 'AGRANINS ',
 'Aibl 1St Islamic Mutual Fund': 'AIBL1STIMF ',
 'Aibl Mudaraba Perpetual Bond': 'AIBLPBOND ',
 'Alif Industries Ltd.': 'AIL ',
 'Al-Haj Textile Mills Ltd.': 'AL-HAJTEX ',
 'Al-Arafah Islami Bank Ltd': 'ALARABANK ',
 'Alif Manufacturing Company Ltd.': 'ALIF ',
 'Alltex Industries Ltd.': 'ALLTEX ',
 'Aman Feed Ltd.': 'AMANFEED ',
 'Ambee Pharmaceuticals  Ltd.': 'AMBEEPHA ',
 'Agricultural Marketing Co

In [4]:
import pandas as pd 

df = pd.read_csv("news_body_new.tsv", sep="\t")

ids = []
dates = []
bodies = []
companies = []
ticks = []

for _, row in df.iterrows():
    try:
        id = row["id"]
        for c in new_l[id]:
            cp = c if c in q.keys() else None
            if cp is not None:
                companies.append(cp)
                ticks.append(q[cp])
                ids.append(id)
                dates.append(row["date"])
                bodies.append(cp + "\n" + row["body"])
    except:
        pass

ab = {
    "id": ids,
    "date": dates,
    "company": companies, 
    "tick": ticks,
    "body": bodies,

}

pf = pd.DataFrame(ab)
pf.to_csv("company_wise_news.tsv", sep="\t", index=False)




In [45]:
import sqlite3

# create new database
conn = sqlite3.connect('stock_table.db')

# create Cursor to execute queries
cur = conn.cursor()
# p = cur.execute("SELECT * FROM stock_5 WHERE tick='ISLAMIBANK'")
# for rwo in p:
#     print(rwo)

def get_data(tick, date):
    cur.execute("SELECT * FROM stock_5 WHERE tick=? and date=?", (tick, date))

    rows = cur.fetchall()
    # print(rows)
    change = -1
    for row in rows:
        _, _, _, change = row
    return change

tick = 'ISLAMIBANK'
date = '25-11-2015'

tick, date = "26-05-2016 BERGERPBL "
get_data(tick, date)
# for row in rows:
#     print(row)
print('Databse created.')

ValueError: too many values to unpack (expected 2)

In [47]:
import pandas as pd 

df = pd.read_csv("company_wise_news.tsv", sep="\t")
# df["change"] = -1
changes = []
for _, row in df.iterrows():
    tick = row["tick"].strip()
    date = row["date"]
    print(date, tick)
    change = get_data(tick, date)
    print(change)
    # row["change"] = change
    changes.append(change)



df["change"] = changes
df

26-05-2016 SPCL
-1
26-05-2016 GP
-1
26-05-2016 BATBC
-1
26-05-2016 BERGERPBL
-1
26-05-2016 LINDEBD
-1
06-07-2015 BXPHARMA
2
06-07-2015 FBFIF
2
21-08-2015 SQURPHARMA
-1
21-08-2015 SPCL
-1
21-08-2015 BBS
-1
21-08-2015 ISLAMIBANK
-1
21-08-2015 AL-HAJTEX
-1
30-08-2016 NTLTUBES
2
30-08-2016 FARCHEM
1
30-08-2016 KEYACOSMET
1
03-04-2015 GP
-1
03-04-2015 SQURPHARMA
-1
03-04-2015 UTTARABANK
-1
12-04-2016 OLYMPIC
2
12-04-2016 KEYACOSMET
1
12-04-2016 AMANFEED
2
12-04-2016 DSHGARME
1
14-10-2016 GP
-1
14-10-2016 PRAGATILIF
-1
14-10-2016 ACTIVEFINE
-1
14-10-2016 AZIZPIPES
-1
14-10-2016 BBS
-1
14-10-2016 BSCCL
-1
14-10-2016 BEACHHATCH
-1
14-10-2016 BXPHARMA
-1
14-10-2016 BXSYNTH
-1
14-10-2016 DAFODILCOM
-1
14-10-2016 FARCHEM
-1
14-10-2016 GHAIL
-1
14-10-2016 IMAMBUTTON
-1
14-10-2016 ISNLTD
-1
14-10-2016 KPPL
-1
14-10-2016 MPETROLEUM
-1
14-10-2016 PADMALIFE
-1
14-10-2016 RDFOOD
-1
14-10-2016 SAIHAMCOT
-1
14-10-2016 SAIHAMTEX
-1
14-10-2016 SALVOCHEM
-1
14-10-2016 SHASHADNIM
-1
14-10-2016 SPCERAMICS
-1


,id,date,company,tick,body,change
0,1bc32b9fc4ca4f288c8b847de4f9b6df,26-05-2016,Shahjibazar Power Co. Ltd.,SPCL,"Shahjibazar Power Co. Ltd.\npital Partners, th...",-1
1,1bc32b9fc4ca4f288c8b847de4f9b6df,26-05-2016,Grameenphone Ltd.,GP,"Grameenphone Ltd.\npital Partners, the Dhaka b...",-1
2,1bc32b9fc4ca4f288c8b847de4f9b6df,26-05-2016,British American Tobacco Bangladesh Company Ltd.,BATBC,British American Tobacco Bangladesh Company Lt...,-1
3,1bc32b9fc4ca4f288c8b847de4f9b6df,26-05-2016,Berger Paints Bangladesh Ltd.,BERGERPBL,"Berger Paints Bangladesh Ltd.\npital Partners,...",-1
4,1bc32b9fc4ca4f288c8b847de4f9b6df,26-05-2016,Linde Bangladesh Ltd.,LINDEBD,"Linde Bangladesh Ltd.\npital Partners, the Dha...",-1
...,...,...,...,...,...,...
1486,a2fc249537f642af881378f8a2a2aa08,11-06-2017,United Power Generation & Distribution Company...,UPGDCL,United Power Generation & Distribution Company...,2
1487,a2fc249537f642af881378f8a2a2aa08,11-06-2017,Paramount Textile Ltd.,PTL,Paramount Textile Ltd.\ncompanies which domina...,1
1488,a2fc249537f642af881378f8a2a2aa08,11-06-2017,Argon Denims Ltd.,ARGONDENIM,Argon Denims Ltd.\ncompanies which dominated t...,1
1489,d9c3cf8cc5a74841862cd83924d5cb5d,27-04-2017,Prime Bank Ltd.,PRIMEBANK,Prime Bank Ltd.\nangla Finance featured a turn...,-1


In [49]:
df = df[df['change']>=0]
df.to_csv("final_data.tsv", sep="\t", index=False)

In [50]:
df

,id,date,company,tick,body,change
5,f6d98dbd22714e96aa3dc762c8ba83fc,06-07-2015,Beximco Pharmaceuticals Ltd.,BXPHARMA,Beximco Pharmaceuticals Ltd.\ntivities on the ...,2
6,f6d98dbd22714e96aa3dc762c8ba83fc,06-07-2015,First Bangladesh Fixed Income Fund,FBFIF,First Bangladesh Fixed Income Fund\ntivities o...,2
12,5623ae1043ab4c4e8ee0537b96600e09,30-08-2016,National Tubes Ltd.,NTLTUBES,National Tubes Ltd.\nturnover on the DSE were ...,2
13,5623ae1043ab4c4e8ee0537b96600e09,30-08-2016,Far Chemical Industries Ltd.,FARCHEM,Far Chemical Industries Ltd.\nturnover on the ...,1
14,5623ae1043ab4c4e8ee0537b96600e09,30-08-2016,Keya Cosmetics Ltd.,KEYACOSMET,Keya Cosmetics Ltd.\nturnover on the DSE were ...,1
...,...,...,...,...,...,...
1484,632fb6c4369a4f59b169d995c303a03e,19-06-2018,Federal Insurance Company Ltd.,FEDERALINS,Federal Insurance Company Ltd.\n18 listed comp...,0
1485,a2fc249537f642af881378f8a2a2aa08,11-06-2017,Beximco Pharmaceuticals Ltd.,BXPHARMA,Beximco Pharmaceuticals Ltd.\ncompanies which ...,1
1486,a2fc249537f642af881378f8a2a2aa08,11-06-2017,United Power Generation & Distribution Company...,UPGDCL,United Power Generation & Distribution Company...,2
1487,a2fc249537f642af881378f8a2a2aa08,11-06-2017,Paramount Textile Ltd.,PTL,Paramount Textile Ltd.\ncompanies which domina...,1
